In [ ]:
from glob import glob

pdf_paths = glob('./pdfs/*.pdf')
pdf_paths

In [ ]:
from pdf2image import convert_from_path
import os

if not os.path.exists('images'):
    os.makedirs('images')
for pdf_path in pdf_paths:
    if not os.path.exists(f'images/{pdf_path[7:-4]}'):
        os.makedirs(f'images/{pdf_path[7:-4]}')
    pages = convert_from_path(pdf_path, 350)
    for i, page in enumerate(pages):
        page.save(f'./images/{pdf_path[7:-4]}/page_{i}.jpg', 'JPEG')

In [ ]:
pdf_name_image_list = {}
for pdf_path in pdf_paths:
    pdf_name_image_list[f'{pdf_path[7:-4]}'] = glob(f'./images/{pdf_path[7:-4]}/*.jpg')
pdf_name_image_list

In [1]:
from pdf2image import convert_from_path

pdfs = r"./doc1.pdf"
pages = convert_from_path(pdfs, 350)

i = 1
for page in pages:
    image_name = "Page_" + str(i) + ".jpg"  
    page.save(image_name, "JPEG")
    i = i+1 

In [2]:
# use this command to install open cv2
# pip install opencv-python

# use this command to install PIL
# pip install Pillow

import cv2
from PIL import Image

def mark_region(image_path):
    
    im = cv2.imread(image_path)

    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9,9), 0)
    thresh = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,30)

    # Dilate to combine adjacent text contours
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
    dilate = cv2.dilate(thresh, kernel, iterations=4)

    # Find contours, highlight text areas, and extract ROIs
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    line_items_coordinates = []
    for c in cnts:
        area = cv2.contourArea(c)
        x,y,w,h = cv2.boundingRect(c)

        if y >= 600 and x <= 1000:
            if area > 10000:
                image = cv2.rectangle(im, (x,y), (2200, y+h), color=(255,0,255), thickness=3)
                line_items_coordinates.append([(x,y), (2200, y+h)])

        if y >= 2400 and x<= 2000:
            image = cv2.rectangle(im, (x,y), (2200, y+h), color=(255,0,255), thickness=3)
            line_items_coordinates.append([(x,y), (2200, y+h)])


    return image, line_items_coordinates


In [ ]:
import cv2
import pytesseract
import matplotlib.pyplot as plt
import os

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Function to process and display each image
def process_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error loading image: {image_path}")
    else:
        ret, thresh1 = cv2.threshold(image, 120, 255, cv2.THRESH_BINARY)
        text = pytesseract.image_to_string(thresh1, config='--psm 6')
        print(f"OCR Results for {image_path}:\n{text}\n")
        img_rgb = cv2.cvtColor(thresh1, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(10,10))
        plt.imshow(img_rgb)
        plt.show()

# List all files in the current directory
files = os.listdir('.')

# Filter for image files (e.g., .jpg, .jpeg, .png)
image_files = [f for f in files if f.endswith(('.jpg', '.jpeg', '.png'))]

# Process each image file
for image_file in image_files:
    process_image(image_file)

In [ ]:
from pdf2image import convert_from_path
import spacy
import pytesseract
import cv2
import numpy as np

# Path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Load the Spanish NLP model
nlp = spacy.load("es_core_news_sm")

# Convert PDF pages to images
pdfs = r"./doc1.pdf"
pages = convert_from_path(pdfs, 350)

# Function to process a single page image and extract text from two columns
def process_image(image_path):
    # Read the image
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    print(f"Image dimensions: {width}x{height}")
    # Divide the image into two columns
    left_col = image[850:3594, :width // 2]
    right_col = image[850:3594, width // 2:]

    # Convert to binary for better OCR results
    ret1, thresh1 = cv2.threshold(left_col, 120, 255, cv2.THRESH_BINARY)
    ret2, thresh2 = cv2.threshold(right_col, 120, 255, cv2.THRESH_BINARY)

    # OCR configuration
    custom_config = r'--oem 3 --psm 4'

    # Extract text from both columns
    text_left = str(pytesseract.image_to_string(thresh1, config=custom_config))
    text_right = str(pytesseract.image_to_string(thresh2, config=custom_config))

    # Combine the text from both columns
    text = text_left + "\n" + text_right

    return text

# Process each page and extract text
for i, page in enumerate(pages):
    image_name = f"Page_{i + 1}.jpg"
    page.save(image_name, "JPEG")

    extracted_text = process_image(image_name)
    print(extracted_text)
    # Process the extracted text with spaCy
    # doc = nlp(extracted_text)
    
    # Print extracted entities
    # for ent in doc.ents:
    #     print(
    #         f"""
    #         {ent.text = }
    #         {ent.start_char = }
    #         {ent.end_char = }
    #         {ent.label_ = }
    #         spacy.explain('{ent.label_}') = {spacy.explain(ent.label_)}"""
    #     )
